In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import csv
import tensorflow as tf
import timeit
import math
from sympy import limit, Symbol
from tensorflow.keras.utils import to_categorical
from pandas import DataFrame
from copy import copy, deepcopy
from beautifultable import BeautifulTable


In [4]:
class Person:
    def __init__(self, id, filename, img, gender, age, angle):
        self.id = id
        self.filename = filename
        self.img = img
        self.gender = gender
        self.age = age
        self.angle = angle
        self.guessGender = 0
        self.guessAge = 0
        self.guessAngle = 0
        self.probability = 0
        
    def printAll(self):
        print('ID: ', self.id,'\n')
        print('Angle: ', self.angle,' Guess Angle: ', self.guessAngle)
        print('Gender: ', self.gender,' Guess Gender: ', self.guessGender)
        print('Age: ', self.age,' Guess Age: ', self.guessAge,'\n')

In [5]:
category_angle = [0, 15, 30, 45, 60, 75, 90, 180, 195, 210, 225, 240, 255, 270]
category_gender = ["F","M"]

In [6]:
%matplotlib inline

df = pd.read_csv('subject_info_OUMVLP.csv')

In [7]:
id_train = np.loadtxt("IDList_train_OUMVLP.txt", dtype='int32')

In [8]:
# The dir stored GEIs
datadir = r"C:/Users/Acer/FYP/GEI_OUMVLP/"
files = os.listdir(datadir)

all_people = []

df.ID = df.ID.astype(str)
for index, data in df.iterrows():
    id = (5-len(data.ID))*'0' + data.ID
    gender = 1 if data.gender == 'M' else 0
    age = data.age
    if age == '-':
        age = '-'
    else:
        age = int(age)
    
    temp_id_list = []
    for file in files:
        angle = int(file.split('-')[0])
        sequence = int(file.split('-')[1])
        
        if sequence == 1:
            if id in temp_id_list :
                temp_id_list = []
                continue
            temp_id_list = []
        
        filename = datadir + file + '/' + id + '.png'
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        if type(img) == type(None):
            continue
        
        p = Person(id, filename, img, gender, age, angle)
        all_people.append(p)
        temp_id_list.append(id)
        print (id, end="\r")


In [9]:
people_train = []
people_test = []

for p in all_people:
    if int(p.id) in id_train:
        people_train.append(p)
    else:
        people_test.append(p)
        
height, width = all_people[0].img.shape

In [27]:
all_people_with_angle = {}

for p in all_people:
    if p.angle not in all_people_with_angle:
        all_people_with_angle[p.angle] = []
    all_people_with_angle[p.angle].append(p)

# Load all the trained block

In [4]:
with tf.device('CPU:0'):
    block_angle = tf.keras.models.load_model('model_MVLP2/MVLP_angle.h5')
    block_gender_0_180 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_0_180.h5')
    block_age_F_0_180 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_0_180.h5')
    block_age_M_0_180 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_0_180.h5')

    block_gender_15_195 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_15_195.h5')
    block_age_F_15_195 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_15_195.h5')
    block_age_M_15_195 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_15_195.h5')

    block_gender_30_45 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_30_45.h5')
    block_age_F_30_45 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_30_45.h5')
    block_age_M_30_45 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_30_45.h5')

    block_gender_60_75_90 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_60_75_90.h5')
    block_age_F_60_75_90 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_60_75_90.h5')
    block_age_M__60_75_90 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_60_75_90.h5')

    block_gender_210_225 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_210_225.h5')
    block_age_F_210_225 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_210_225.h5')
    block_age_M_210_225 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_210_225.h5')

    block_gender_240_255_270 = tf.keras.models.load_model('model_MVLP2/MVLP_gender_240_255_270.h5')
    block_age_F_240_255_270 = tf.keras.models.load_model('model_MVLP2/MVLP_age_F_240_255_270.h5')
    block_age_M_240_255_270 = tf.keras.models.load_model('model_MVLP2/MVLP_age_M_240_255_270.h5')

In [5]:
MVLP_model = {}

MVLP_model = {'model':block_angle}
MVLP_model[0,180] = {'model':block_gender_0_180, 0:block_age_F_0_180, 1:block_age_M_0_180}
MVLP_model[15,195] = {'model':block_gender_15_195, 0:block_age_F_15_195, 1:block_age_M_15_195}
MVLP_model[30,45] = {'model':block_gender_30_45, 0:block_age_F_30_45, 1:block_age_M_30_45}
MVLP_model[60,75,90] = {'model':block_gender_60_75_90, 0:block_age_F_60_75_90, 1:block_age_M__60_75_90}
MVLP_model[210,225] = {'model':block_gender_210_225, 0:block_age_F_210_225, 1:block_age_M_210_225}
MVLP_model[240,255,270] = {'model':block_gender_240_255_270, 0:block_age_F_240_255_270, 1:block_age_M_240_255_270}



In [6]:
temp = {}

for ks in MVLP_model:
    if type(ks) == type(tuple()):
        for k in ks:
            temp[k] = MVLP_model[ks]
    else:
        temp[ks] = MVLP_model[ks]

MVLP_model = temp
del temp
print(MVLP_model.keys())
            

dict_keys(['model', 0, 180, 15, 195, 30, 45, 60, 75, 90, 210, 225, 240, 255, 270])


# Predict the GEIs

In [ ]:
with tf.device("CPU:0"):
    start = timeit.default_timer()
    for p in all_people:
        img = np.array(p.img).reshape(-1, height, width, 1)
        predict_angle = MVLP_model['model'].predict(img)
        p.guessAngle = category_angle[np.argmax(predict_angle)]

        predict_gender = MVLP_model[p.guessAngle]['model'].predict(img)
        p.guessGender = np.argmax(predict_gender)

        if p.age != '-':
            predict_age = MVLP_model[p.guessAngle][p.guessGender].predict(img)
            p.guessAge = np.argmax(predict_age)+2

        b = "ID: "+ p.id + " Guess: " + str(p.guessAngle) + "  " + category_gender[p.guessGender] + " " + str(p.guessAge) + " year-old                                 "
        print (b, end="\r")

    stop = timeit.default_timer()

In [ ]:
s = stop - start
hours = ((stop - start)/60)/60
mins = hours*60 - int(hours)*60

print('Prediction Time: ', round(hours), 'hours', round(mins), 'mins')

In [ ]:
#Prediction Time:  5 hours 25 mins

# Saving results to a .csv file

In [ ]:
# Filtering the age with '-'

for p in all_people:
    if p.age == '-':
        p.guessAge = '-'
        

In [ ]:
col = ['id', 'filename', 'gender', 'age', 'angle', 'guessGender', 'guessAge', 'guessAngle', 'probability']
pred_prople = []
for p in all_people:
    filename = p.filename.split('/FYP/')[1]
    pred_prople.append([p.id,filename,p.gender,p.age,p.angle,p.guessGender,p.guessAge,p.guessAngle])

df_pred_prople = DataFrame(pred_prople, columns = col)

In [ ]:
df_pred_prople.to_csv('model_MVLP2/Predicted_Result_MVLP.csv',index=0)

# Truth table of angle

In [16]:
truth_table_angle = {}
df_angle = []
total = [0] * (len(category_angle) + 1)

for c in category_angle:
    truth_table_angle[c] = {}
    for c2 in category_angle:
        truth_table_angle[c][c2] = 0
        
for p in people_test:
    truth_table_angle[p.guessAngle][p.angle]+=1

for angle in truth_table_angle:
    temp = []
    index = 0
    count = 0
    for key in truth_table_angle[angle]:
        temp.append(truth_table_angle[angle][key])
        count+=truth_table_angle[angle][key]
        total[index]+=truth_table_angle[angle][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_angle.append(temp)
df_angle.append(total)

In [17]:
columns = deepcopy(category_angle)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_angle)):
    df_angle[i].insert(0,columns[i+1])
    table.rows.append(df_angle[i])

table.maxwidth = 150
print(table)

+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+
| Truth --> |  0   |  15  |  30  |  45  |  60  |  75  |  90  | 180  | 195  | 210  | 225  | 240  | 255  | 270  | Total |
|           |      |      |      |      |      |      |      |      |      |      |      |      |      |      |       |
|  Predict  |      |      |      |      |      |      |      |      |      |      |      |      |      |      |       |
+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+
|     0     | 4655 |  4   |  0   |  0   |  0   |  0   |  1   |  57  |  0   |  0   |  0   |  0   |  0   |  0   | 4717  |
+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+
|    15     |  7   | 4790 |  8   |  1   |  0   |  0   |  0   |  4   |  6   |  0   |  0   |  0   |  0   |  0   | 4816  |
+-----------+------+------+------+------

# Truth table of gender

In [23]:
truth_table_gender = {}
df_gender = []
total = [0] * (len(category_gender) + 1)

for c in category_gender:
    truth_table_gender[c] = {}
    for c2 in category_gender:
        truth_table_gender[c][c2] = 0
        
for p in all_people:
    truth_table_gender[category_gender[p.gender]][category_gender[p.guessGender]]+=1

for gender in truth_table_gender:
    temp = []
    index = 0
    count = 0
    for key in truth_table_gender[gender]:
        temp.append(truth_table_gender[gender][key])
        count+=truth_table_gender[gender][key]
        total[index]+=truth_table_gender[gender][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_gender.append(temp)
df_gender.append(total)

In [25]:
columns = deepcopy(category_gender)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_gender)):
    df_gender[i].insert(0,columns[i+1])
    table.rows.append(df_gender[i])

table.maxwidth = 150
print(table)

+-----------+-------+-------+--------+
| Truth --> |   F   |   M   | Total  |
|           |       |       |        |
|  Predict  |       |       |        |
+-----------+-------+-------+--------+
|     F     | 67923 | 1804  | 69727  |
+-----------+-------+-------+--------+
|     M     | 1500  | 66587 | 68087  |
+-----------+-------+-------+--------+
|   Total   | 69423 | 68391 | 137814 |
+-----------+-------+-------+--------+


# Eveluate age

In [28]:
people_with_value_age = {}
for angle in all_people_with_angle:
    people_with_value_age[angle] = []
    for p in all_people_with_angle[angle]:
        if p.age != "-":
            people_with_value_age[angle].append(p)

In [29]:
MAE = {}
MAE['angle']={}
for c in category:
    MAE['angle'][c]=0


In [30]:
for c in people_with_value_age:
    for p in people_with_value_age[c]:
        MAE['angle'][c]+=abs(int(p.age)-int(p.guessAge))
    MAE['angle'][c] = round(MAE['angle'][c]/len(people_with_value_age[c]),2)

In [31]:
mean = 0
for c in MAE['angle']:
    mean += MAE['angle'][c]
mean = round(mean/len(MAE['angle']),2)
MAE['angle']['mean'] = mean
print(MAE)

{'angle': {0: 8.99, 15: 9.02, 30: 5.1, 45: 5.05, 60: 5.16, 75: 4.82, 90: 4.96, 180: 9.53, 195: 8.92, 210: 5.07, 225: 5.01, 240: 5.24, 255: 5.06, 270: 5.18, 'mean': 6.22}}


In [32]:
Metrics = []
for key in MAE['angle']:
    Metrics.append(MAE['angle'][key])

columns = deepcopy(category)
columns.append('Mean')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
table.rows.append(Metrics)
table.rows.header = ["MAE"]
table.maxwidth = 150
print(table)

+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+
|     |  0   |  15  | 30  |  45  |  60  |  75  |  90  | 180  | 195  | 210  | 225  | 240  | 255  | 270  | Mean |
+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+
| MAE | 8.99 | 9.02 | 5.1 | 5.05 | 5.16 | 4.82 | 4.96 | 9.53 | 8.92 | 5.07 | 5.01 | 5.24 | 5.06 | 5.18 | 6.22 |
+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+


# Percentage of each results

In [ ]:
corr_count_angle = 0
corr_count_gender = 0
corr_count_age = 0
all_corr = 0

for p in people_test:
    count = 0
    if p.angle == p.guessAngle:
        corr_count_angle+=1
        count+=1
    
    if p.gender == p.guessGender:
        corr_count_gender+=1
        count+=1
    
    if p.age == p.guessAge:
        corr_count_age+=1
        count+=1
        
    if count == 3:
        all_corr+=1
         
acc_angle = corr_count_angle/len(people_test)
acc_gender = corr_count_gender/len(people_test)
acc_age = corr_count_age/len(people_test)
acc_all = all_corr/len(people_test)
avg_acc = (acc_angle+acc_gender+acc_age)/3


In [ ]:
print("Accuracy of Predicting angle is: " + str(round(acc_angle*100,2)) + '%')
print("Accuracy of Predicting gender is: " + str(round(acc_gender*100,2)) + '%')
print("Accuracy of Predicting age is: " + str(round(acc_age*100,2)) + '%','\n')

print("Accuracy of Three items passed at the same time: " + str(round(acc_all*100,2)) + '%')
print("The Average Accuracy: " + str(round(avg_acc*100,2)) + '%')

# Calculate SD

In [39]:
type(MAE['angle']['mean'])

float

In [43]:
sumation = 0
for p in filted:
    sumation+=(abs(int(p.age)-p.guessAge) - MAE['angle']['mean']) ** 2

SD = math.sqrt(sumation/(len(all_people)-1))

In [44]:
print(SD)

10.358128437573736


# Calculate MCE

In [13]:
len(filted)

134951

In [14]:
for p in filted:
    p.age = int(p.age)
    p.img = cv2.imread(p.filename, cv2.IMREAD_GRAYSCALE)
    
    
    

In [15]:
height, width = filted[0].img.shape

In [17]:
start = timeit.default_timer()
for p in filted:
    img = np.array(p.img).reshape(-1, height, width, 1)
    
    predict_angle = MVLP_model['model'].predict(img)
    ga = category_angle[np.argmax(predict_angle)]

    predict_gender = MVLP_model[ga]['model'].predict(img)
    gg = np.argmax(predict_gender)

    predict_age = MVLP_model[ga][gg].predict(img)
    p.probability = predict_age[0][p.age-2]

    b = p.id + " probability: " + str(predict_age[0][p.age-2]) + "                                 "
    print (b, end="\r")

stop = timeit.default_timer()

In [18]:
s = stop - start
hours = ((stop - start)/60)/60
mins = hours*60 - int(hours)*60

print('Prediction Time: ', round(hours), 'hours', round(mins), 'mins')

Prediction Time:  4 hours 19 mins


In [113]:
sumation = 0
count = 0
for proba in estimated_probability:
    if proba == 0:
        count+=1
        continue
    sumation+=math.log(proba,10)


In [115]:
MCE = sumation/(len(estimated_probability)-count)
print(count)
print(MCE)

30
-1.9196574810141178


In [118]:
estimated_probability_test = []
start = timeit.default_timer()
for p in filted:
    if int(p.id) in id_train:
        continue
    
    img = np.array(p.img).reshape(-1, height, width, 1)
    
    predict_angle = MVLP_model['model'].predict(img)
    ga = category_angle[np.argmax(predict_angle)]

    predict_gender = MVLP_model[ga]['model'].predict(img)
    gg = np.argmax(predict_gender)

    predict_age = MVLP_model[ga][gg].predict(img)
    estimated_probability_test.append(predict_age[0][p.age-2])

    b = p.id + " probability: " + str(predict_age[0][p.age-2]) + "                                 "
    print (b, end="\r")

stop = timeit.default_timer()

In [120]:
sumation = 0
count = 0
for proba in estimated_probability_test:
    if proba == 0:
        count+=1
        continue
    sumation+=math.log(proba,10)

In [121]:
MCE = sumation/(len(estimated_probability)-count)
print(count)
print(MCE)

20
-1.7772965252917137


In [126]:
count = 0
for p in all_people:
    if p.gender == p.guessGender:
        count += 1
        
CCR = count/len(all_people)
print(CCR)

0.9760256577706183


In [130]:
count = 0
for p in people_test:
    if p.gender == p.guessGender:
        count += 1
        
CCR = count/len(people_test)
print(CCR)

0.9521408389480548


# Calculate CS

In [144]:
cs1 = 0
cs5 = 0
cs10 = 0
count = 0

for p in filted:
    if int(p.id) in id_train:
        continue
    
    count+=1
    
    if abs(p.age-p.guessAge) <= 1:
        cs1+=1
    
    if abs(p.age-p.guessAge) <= 5:
        cs5+=1
        
    if abs(p.age-p.guessAge) <= 10:
        cs10+=1
    
cs1 = (cs1/count)*100
cs5 = (cs5/count)*100
cs10 = (cs10/count)*100

In [145]:
print(round(cs1,2),'%')
print(round(cs5,2),'%')
print(round(cs10,2),'%')

22.52 %
48.33 %
64.18 %


In [148]:
table = BeautifulTable()
table.rows.append(['Method', 'MAE', "CS(1)", "CS(5)", "CS(10)", 'MCE', 'CCR'])
table.rows.append(['GEINet', '9.11', '15.70', '46.73', '65.82', '-13.60', 'N/A'])
table.rows.append(['GaitSet', '9.02', '16.39', '47.59', '66.01', '-5.04', '92.72'])
table.rows.append(["Xu's Method", '8.39', '15.84', '48.00', '68.40', '-4.27', '94.27'])
table.rows.append(["Our Method", '6.22', '22.52', '48.33', '64.18', '-1.78', '95.21'])

print(table)

+-------------+------+-------+-------+--------+-------+-------+
|   Method    | MAE  | CS(1) | CS(5) | CS(10) |  MCE  |  CCR  |
+-------------+------+-------+-------+--------+-------+-------+
|   GEINet    | 9.11 | 15.7  | 46.73 | 65.82  | -13.6 |  N/A  |
+-------------+------+-------+-------+--------+-------+-------+
|   GaitSet   | 9.02 | 16.39 | 47.59 | 66.01  | -5.04 | 92.72 |
+-------------+------+-------+-------+--------+-------+-------+
| Xu's Method | 8.39 | 15.84 | 48.0  |  68.4  | -4.27 | 94.27 |
+-------------+------+-------+-------+--------+-------+-------+
| Our Method  | 6.22 | 22.52 | 48.33 | 64.18  | -1.78 | 95.21 |
+-------------+------+-------+-------+--------+-------+-------+


In [2]:
table = BeautifulTable()
table.rows.append(['Groups', 'Contained Angles'])


print(table)

+--------+------------------+
| Groups | Contained Angles |
+--------+------------------+
|   1    |      0, 180      |
+--------+------------------+
|   2    |     15, 195      |
+--------+------------------+
|   3    |      30, 45      |
+--------+------------------+
|   4    |    60, 75, 90    |
+--------+------------------+
|   5    |     210, 225     |
+--------+------------------+
|   6    |  240, 255, 270   |
+--------+------------------+


In [4]:
table = BeautifulTable()
table.rows.append(['Layer', 'Output dimension', 'Kernel size', 'Strides','Units'])

table.rows.append(['Conv 2D a', '18', '7x7', '1', '-'])
table.rows.append(['Max Pooling a', '-', '2x2', '2', '-'])
table.rows.append(['Batch Norm a', '-', '-', '-', '-'])

table.rows.append(['Conv 2D b', '45', '5x5', '1', '-'])
table.rows.append(['Max Pooling b', '-', '3x3', '2', '-'])
table.rows.append(['Batch Norm b', '-', '-', '-', '-'])

table.rows.append(['Conv 2D c', '60', '3x3', '1', '-'])
table.rows.append(['Max Pooling c', '-', '2x2', '2', '-'])
table.rows.append(['Batch Norm c', '-', '-', '-', '-'])

table.rows.append(['Flatten Layer', '-', '-', '-', '-'])
table.rows.append(['Densely-Connected', '-', '-', '-', '1024'])
table.rows.append(['Batch Norm d', '-', '-', '-', '-'])
table.rows.append(['Dropout Layer', '-', '-', '-', '-'])

table.rows.append(['Output Layer', '-', '-', '-', '2'])

#table.maxwidth = 150
print(table)

+-------------------+------------------+-------------+---------+-------+
|       Layer       | Output dimension | Kernel size | Strides | Units |
+-------------------+------------------+-------------+---------+-------+
|     Conv 2D a     |        18        |     7x7     |    1    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Max Pooling a   |        -         |     2x2     |    2    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Batch Norm a    |        -         |      -      |    -    |   -   |
+-------------------+------------------+-------------+---------+-------+
|     Conv 2D b     |        45        |     5x5     |    1    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Max Pooling b   |        -         |     3x3     |    2    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Batch Norm b    |        -         |      -    

# MCE by angle

In [27]:
category = [0, 15, 30, 45, 60, 75, 90, 180, 195, 210, 225, 240, 255, 270]

all_people_with_angle_filted = {}

for p in filted:
    if p.angle not in all_people_with_angle_filted:
        all_people_with_angle_filted[p.angle] = []
    all_people_with_angle_filted[p.angle].append(p)